AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [39]:
# !pip install autogluon.timeseries

In [40]:
# Importar librerías
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [41]:
# Cargar datasets
df_sellin = pd.read_csv("../data/sell-in.txt", sep="\t")
df_productos = pd.read_csv("../data/tb_productos.txt", sep="\t")

# Leer lista de productos a predecir
with open("../data/product_id_apredecir201912.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [42]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

In [43]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [44]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [45]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [46]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [47]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [10]:
# Filter out unsupported models and convert to dictionary format
supported_models = ['DeepAR', 'PatchTST', 'TemporalFusionTransformer']
hyperparameters_dict = {model: {} for model in supported_models}

predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'
)

predictor.fit(
    ts_data, 
    num_val_windows=2, 
    time_limit=60*60, 
    presets=None, 
    hyperparameters=hyperparameters_dict,
    enable_ensemble=True
)


Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to '/home/nespina/Documentos/austral/labo_3/src/AutogluonModels/ag-20250719_015202'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #66-Ubuntu SMP PREEMPT_DYNAMIC Fri Jun 13 20:25:30 UTC 2025
CPU Count:          8
GPU Count:          0
Memory Avail:       4.89 GB / 11.37 GB (43.0%)
Disk Space Avail:   107.92 GB / 284.85 GB (37.9%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': {'DeepAR': {},
                     'PatchTST': {},
                     'TemporalFusionTransformer': {}},
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': Fa

In [11]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [12]:
# Extraer predicción media y filtrar febrero 2019
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [13]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2019
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2019-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


In [50]:
resultado.head()

,product_id,tn
1,20001,1254.235273
3,20002,904.599322
5,20003,722.232125
7,20004,503.424966
9,20005,381.429058


In [51]:
# Creamos un "slicer" para seleccionar en el índice multinivel
idx = pd.IndexSlice

# Seleccionamos todos los item_id (usando ':') y filtramos el timestamp para '2019-02'
df_febrero_2019 = ts_data.loc[idx[:, '2019-02'], :]

# Ahora 'df_febrero_2019' contiene los datos que necesitas
print(df_febrero_2019.head())

                    product_id          tn
item_id timestamp                         
20001   2019-02-01       20001  1259.09363
20002   2019-02-01       20002  1043.01349
20003   2019-02-01       20003   758.32657
20004   2019-02-01       20004   441.70332
20005   2019-02-01       20005   409.89950


In [54]:
df_febrero_2019.head()

,,product_id,tn
item_id,timestamp,,
20001,2019-02-01,20001,1259.09363
20002,2019-02-01,20002,1043.01349
20003,2019-02-01,20003,758.32657
20004,2019-02-01,20004,441.70332
20005,2019-02-01,20005,409.89950


In [56]:
# Crear un DataFrame con los coeficientes para cada product_id en resultado
# 1. Hacemos merge para alinear los tn de resultado y df_febrero_2019
# 2. Calculamos el coeficiente como tn_real / tn_predicho
# 3. Para los que no existen en df_febrero_2019, el coeficiente será 1

# Convertir df_febrero_2019 a DataFrame plano para merge
df_febrero_2019_reset = df_febrero_2019.reset_index()[['product_id', 'tn']].rename(columns={'tn': 'tn_real'})

# Unir resultado con los valores reales
coef_df = resultado.merge(df_febrero_2019_reset, on='product_id', how='left')

# Calcular coeficiente
coef_df['coef'] = coef_df['tn_real'] / coef_df['tn']

# Para los que no existen en df_febrero_2019, poner coef=1
coef_df['coef'] = coef_df['coef'].fillna(1)

# Dejar solo product_id y coef
coeficientes = coef_df[['product_id', 'coef']]

# Mostrar los primeros coeficientes
coeficientes.head()

,product_id,coef
0,20001,1.003874
1,20002,1.153012
2,20003,1.049976
3,20004,0.877397
4,20005,1.074642


In [57]:
# 💾 7. Guardar archivo
coeficientes.to_csv("coeficientes.csv", index=False)
coeficientes.head()

,product_id,coef
0,20001,1.003874
1,20002,1.153012
2,20003,1.049976
3,20004,0.877397
4,20005,1.074642


In [58]:
predicciones_febrero2020 = pd.read_csv("predicciones_febrero2020_fecha_01_07.csv")
predicciones_febrero2020.head()

,product_id,tn
0,20001,1277.140118
1,20002,982.076050
2,20003,690.176979
3,20004,509.674443
4,20005,541.772269


In [59]:
# Unir predicciones con coeficientes usando product_id
predicciones_febrero2020 = predicciones_febrero2020.merge(coeficientes, on='product_id', how='left')

# Multiplicar tn por coef (si no hay coef, se asume 1)
predicciones_febrero2020['coef'] = predicciones_febrero2020['coef'].fillna(1)
predicciones_febrero2020['tn'] = predicciones_febrero2020['tn'] * predicciones_febrero2020['coef']

# Eliminar columna coef si no la necesitas más
predicciones_febrero2020 = predicciones_febrero2020.drop(columns=['coef'])

predicciones_febrero2020.head()

,product_id,tn
0,20001,1282.087199
1,20002,1132.345054
2,20003,724.669428
3,20004,447.186590
4,20005,582.210971


In [60]:
predicciones_febrero2020.to_csv("predicciones_febrero2020_ajustadas.csv", index=False)

In [ ]:
# 💾 7. Guardar archivo
resultado.to_csv("predicciones_febrero2020_fecha_01_07.csv", index=False)
resultado.head()

In [ ]:
# Mostrar los mejores modelos del predictor
print("Mejores modelos entrenados:")
print(predictor.leaderboard())